<a href="https://colab.research.google.com/github/TriKhaiLe/Zark-Chat-Server/blob/main/ProcessingAppointmentRequestsFromChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline

# Tạo pipeline NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Thử dự đoán
test_sentence = "Sáng ngày mai họp 8h"
tokens = word_tokenize(test_sentence)
result = ner_pipeline(tokens)
print("NER result:", result)

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


NER result: [[{'entity_group': 'TIME', 'score': np.float32(0.5581903), 'word': 'Sáng', 'start': None, 'end': None}], [], [{'entity_group': 'TIME', 'score': np.float32(0.5655359), 'word': 'họp', 'start': None, 'end': None}], [{'entity_group': 'TIME', 'score': np.float32(0.5036981), 'word': '8', 'start': None, 'end': None}], [{'entity_group': 'TIME', 'score': np.float32(0.5189983), 'word': 'h', 'start': None, 'end': None}]]


In [ ]:
model.save_pretrained("phobert-ner-test")
tokenizer.save_pretrained("phobert-ner-test")
print("Model saved!")

Model saved!


In [ ]:
del model, tokenizer
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("phobert-ner-test")
model = AutoModelForTokenClassification.from_pretrained("phobert-ner-test")
print("Model reloaded successfully!")

NameError: name 'model' is not defined

In [ ]:
del model, tokenizer
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig

# Load the configuration from the saved model
config = AutoConfig.from_pretrained("phobert-ner-test")

# Create the tokenizer
tokenizer = AutoTokenizer.from_pretrained("phobert-ner-test")

# Create the model with the correct number of labels
model = AutoModelForTokenClassification.from_pretrained(
    "phobert-ner-test",
    config=config  # Pass the loaded configuration
)

print("Model reloaded successfully!")


NameError: name 'model' is not defined

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("phobert-ner-test")
print("Số lượng nhãn trong mô hình đã lưu:", config.num_labels)


Số lượng nhãn trong mô hình đã lưu: 7


In [ ]:
ls phobert-ner-test


added_tokens.json  config.json        special_tokens_map.json  vocab.txt
bpe.codes          model.safetensors  tokenizer_config.json


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("phobert-ner-test")

# Bỏ qua mismatch kích thước
model = AutoModelForTokenClassification.from_pretrained("phobert-ner-test", ignore_mismatched_sizes=True)

print("Model loaded successfully with adjusted classifier!")


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at phobert-ner-test and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([7]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully with adjusted classifier!


In [ ]:
del model, tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Tải tokenizer và mô hình gốc
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModelForTokenClassification.from_pretrained("vinai/phobert-base", num_labels=7)

# Định nghĩa nhãn
label_list = ["O", "B-TIME", "I-TIME", "B-EVENT", "I-EVENT", "B-DURATION", "I-DURATION"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}
model.config.label2id = label2id
model.config.id2label = id2label
model.config.num_labels = len(label_list)

# Lưu mô hình
model.save_pretrained("phobert-ner-test")
tokenizer.save_pretrained("phobert-ner-test")
print("Model saved with correct configuration!")

# Tải lại để kiểm tra
del model, tokenizer
tokenizer = AutoTokenizer.from_pretrained("phobert-ner-test")
model = AutoModelForTokenClassification.from_pretrained("phobert-ner-test")
print("Model reloaded successfully!")

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model saved with correct configuration!
Model reloaded successfully!


In [ ]:
print("Số lượng nhãn trong mô hình:", model.config.num_labels)
print("Danh sách nhãn:", model.config.id2label)


NameError: name 'model' is not defined

In [ ]:
text = "Ngày mai tôi sẽ đi Hà Nội lúc 9 giờ sáng."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
print(outputs.logits)  # Logits chưa có ý nghĩa nếu chưa huấn luyện


tensor([[[-0.2098,  0.1362,  0.4154, -0.5087,  0.4963, -0.0928, -0.1592],
         [-0.6076,  0.0734,  0.0674, -0.0802,  0.1577, -0.0253, -0.1013],
         [-0.4290, -0.1553,  0.0866, -0.4612, -0.0012, -0.0977, -0.1074],
         [ 0.0220, -0.3564,  0.0069, -0.0207,  0.0280, -0.4064, -0.1794],
         [-0.0798, -0.1456,  0.0772,  0.1981,  0.2946, -0.1854, -0.0050],
         [-0.3687, -0.0939,  0.0087, -0.1827,  0.3499, -0.2999, -0.0146],
         [-0.1004, -0.4147,  0.2746, -0.0801,  0.1354,  0.1373,  0.0323],
         [-0.4728, -0.3518,  0.3215, -0.2209,  0.2059,  0.2162, -0.0496],
         [-0.0588, -0.1141,  0.1919, -0.2511,  0.0540,  0.0090, -0.2791],
         [-0.0744,  0.1010,  0.1782, -0.2301,  0.3350,  0.0102,  0.0648],
         [-0.1867,  0.0019,  0.0788, -0.3673,  0.0150, -0.0628,  0.2829],
         [-0.2864, -0.0287,  0.2723, -0.0334,  0.0081,  0.0235, -0.0637],
         [-0.2384,  0.2650,  0.1466, -0.3685,  0.7124,  0.1179, -0.2889],
         [-0.2457,  0.0814,  0.3665, -

In [ ]:
import os

# Kiểm tra thư mục chứa model
model_path = "/content/phobert-ner-test"
if os.path.exists(model_path):
    print(f"Model được lưu tại: {model_path}")
    print("Danh sách tệp:")
    print(os.listdir(model_path))
else:
    print("Model chưa được lưu hoặc đường dẫn sai!")


Model chưa được lưu hoặc đường dẫn sai!


In [ ]:
!zip -r phobert-ner-test.zip phobert-ner-test
from google.colab import files
files.download("phobert-ner-test.zip")


	zip warning: name not matched: phobert-ner-test

zip error: Nothing to do! (try: zip -r phobert-ner-test.zip . -i phobert-ner-test)


FileNotFoundError: Cannot find file: phobert-ner-test.zip

In [ ]:
import os

# Kiểm tra thư mục chứa model
model_path = "/content/phobert-ner-test"
if os.path.exists(model_path):
    print(f"Model được lưu tại: {model_path}")
    print("Danh sách tệp:")
    print(os.listdir(model_path))
else:
    print("Model chưa được lưu hoặc đường dẫn sai!")


Model được lưu tại: /content/phobert-ner-test
Danh sách tệp:
['special_tokens_map.json', 'tokenizer_config.json', 'added_tokens.json', 'bpe.codes', 'vocab.txt', 'model.safetensors', 'config.json']


In [ ]:
# !pip install --upgrade transformers
!pip install transformers==4.48.0

import transformers

print(f"Transformers version: {transformers.__version__}")
from transformers import Trainer

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./phobert-ner-results-continued",  # Thư mục lưu kết quả mới
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,                    # Có thể giảm learning rate để tinh chỉnh
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,                    # Số epoch mới để huấn luyện thêm
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

Transformers version: 4.48.0


# Test underthesea

In [ ]:
from underthesea import word_tokenize

text = "Tôi sẽ đi du lịch vào ngày mai."
tokens = word_tokenize(text)

print(tokens)


['Tôi', 'sẽ', 'đi', 'du lịch', 'vào', 'ngày mai', '.']


# Kiểm tra phân bố nhãn

In [ ]:
from collections import Counter
all_labels = [label for sentence in dataset["ner_tags"] for label in sentence]
label_counts = Counter(all_labels)
print("Phân bố nhãn:", label_counts)

  Using cached underthesea-6.8.4-py3-none-any.whl.metadata (15 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached python_crfsuite-0.9.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached underthesea_core-1.0.4-cp311-cp311-manylinux2010_x86_64.whl.metadata (1.7 kB)
  Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached charset_normalizer-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x8

# Load Unthethesea from Drive


In [ ]:
import sys
sys.path.append("/content/drive/My Drive/underthesea_lib")

from underthesea import word_tokenize
print(word_tokenize("Tôi sẽ đi du lịch vào ngày mai."))


ModuleNotFoundError: No module named 'underthesea'

# ***Load Model***

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Tải tokenizer và mô hình gốc
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModelForTokenClassification.from_pretrained("vinai/phobert-base", num_labels=7)

# Định nghĩa nhãn
label_list = ["O", "B-TIME", "I-TIME", "B-EVENT", "I-EVENT", "B-DURATION", "I-DURATION"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}
model.config.label2id = label2id
model.config.id2label = id2label
model.config.num_labels = len(label_list)

# Lưu mô hình
model.save_pretrained("phobert-ner-test")
tokenizer.save_pretrained("phobert-ner-test")
print("Model saved with correct configuration!")

# Tải lại để kiểm tra
del model, tokenizer
tokenizer = AutoTokenizer.from_pretrained("phobert-ner-test")
model = AutoModelForTokenClassification.from_pretrained("phobert-ner-test")
print("Model reloaded successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model saved with correct configuration!
Model reloaded successfully!


# *Train Model*

In [ ]:
!pip install transformers underthesea datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requi

In [ ]:
from google.colab import files
uploaded = files.upload()  # Chọn file train.txt từ máy

Saving train.txt to train.txt


In [ ]:
from datasets import Dataset

def load_data(file_path):
    sentences = []
    tags = []
    current_sentence = []
    current_tags = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == "[END]":
                if current_sentence:
                    sentences.append(current_sentence)
                    tags.append(current_tags)
                    current_sentence = []
                    current_tags = []
            elif line:  # Bỏ qua dòng trống
                token, tag = line.split()
                current_sentence.append(token)
                current_tags.append(tag)

    # Thêm câu cuối nếu có
    if current_sentence:
        sentences.append(current_sentence)
        tags.append(current_tags)

    return Dataset.from_dict({"tokens": sentences, "ner_tags": tags})

dataset = load_data("/content/train.txt")
print(dataset)
print("Sample:", dataset[0])

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 36
})
Sample: {'tokens': ['Sáng', 'ngày_mai', 'họp', '8h', 'trong_vòng', '30p'], 'ner_tags': ['B-TIME', 'I-TIME', 'B-EVENT', 'B-TIME', 'O', 'B-DURATION']}


In [ ]:
from transformers import AutoTokenizer

# Tải tokenizer từ mô hình đã lưu
tokenizer = AutoTokenizer.from_pretrained("phobert-ner-test")

def tokenize_and_align_labels(examples):
    # Tokenize dữ liệu
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,  # Dữ liệu đã là danh sách các từ
        padding="max_length",
        max_length=128,  # Điều chỉnh độ dài tối đa nếu cần
        return_offsets_mapping=False  # Không cần offset mapping với slow tokenizer
    )

    labels = []
    for i, original_labels in enumerate(examples["ner_tags"]):
        # Lấy danh sách token sau khi tokenize cho câu hiện tại
        input_ids = tokenized_inputs["input_ids"][i]
        tokens = tokenizer.convert_ids_to_tokens(input_ids)

        label_ids = []
        word_idx = 0  # Chỉ số từ trong câu gốc

        for token in tokens:
            if token in ["[CLS]", "[SEP]", "[PAD]"]:  # Token đặc biệt
                label_ids.append(-100)
            elif token.startswith("##"):  # Subword tiếp theo của từ
                # Nếu nhãn gốc là I-xxx, giữ nguyên, còn lại thì gán -100
                if word_idx < len(original_labels) and original_labels[word_idx].startswith("I-"):
                    label_ids.append(label2id[original_labels[word_idx]])
                else:
                    label_ids.append(-100)
            else:  # Token đầu tiên của từ
                if word_idx < len(original_labels):
                    label_ids.append(label2id[original_labels[word_idx]])
                    word_idx += 1  # Chuyển sang từ tiếp theo trong câu gốc
                else:
                    label_ids.append(-100)

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Áp dụng tokenize và căn chỉnh nhãn cho tập dữ liệu
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)
print("Tokenized sample:", tokenized_dataset[0])

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Tokenized sample: {'tokens': ['Sáng', 'ngày_mai', 'họp', '8h', 'trong_vòng', '30p'], 'ner_tags': ['B-TIME', 'I-TIME', 'B-EVENT', 'B-TIME', 'O', 'B-DURATION'], 'input_ids': [0, 1316, 5183, 673, 11089, 40201, 323, 5315, 2380, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mas

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("phobert-ner-test")
data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir="./ner_model",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
trainer.save_model("/content/drive/My Drive/ner_model_final")
tokenizer.save_pretrained("/content/drive/My Drive/ner_model_final")
print("Training completed!")

KeyboardInterrupt: 

# Download Model to Device

In [ ]:
!zip -r phobert-ner-test.zip phobert-ner-test
from google.colab import files
files.download("phobert-ner-test.zip")


  adding: phobert-ner-test/ (stored 0%)
  adding: phobert-ner-test/special_tokens_map.json (deflated 57%)
  adding: phobert-ner-test/tokenizer_config.json (deflated 77%)
  adding: phobert-ner-test/added_tokens.json (stored 0%)
  adding: phobert-ner-test/bpe.codes (deflated 59%)
  adding: phobert-ner-test/vocab.txt (deflated 55%)
  adding: phobert-ner-test/model.safetensors


zip error: Interrupted (aborting)


FileNotFoundError: Cannot find file: phobert-ner-test.zip

# Connect with Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


ValueError: mount failed

# Download model to Drive

In [ ]:
model.save_pretrained("/content/drive/My Drive/phobert-ner-test")
tokenizer.save_pretrained("/content/drive/My Drive/phobert-ner-test")


('/content/drive/My Drive/phobert-ner-test/tokenizer_config.json',
 '/content/drive/My Drive/phobert-ner-test/special_tokens_map.json',
 '/content/drive/My Drive/phobert-ner-test/vocab.txt',
 '/content/drive/My Drive/phobert-ner-test/bpe.codes',
 '/content/drive/My Drive/phobert-ner-test/added_tokens.json')

# Config Training Arguments

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)
print("Training arguments configured!")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training arguments configured!


# ***Full Flow***

In [ ]:
# Cài đặt thư viện (chỉ chạy một lần ở đầu nếu cần)
!pip install transformers underthesea datasets

from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import Dataset

# Khởi tạo và lưu mô hình
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModelForTokenClassification.from_pretrained("vinai/phobert-base", num_labels=7)

label_list = ["O", "B-TIME", "I-TIME", "B-EVENT", "I-EVENT", "B-DURATION", "I-DURATION"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}
model.config.label2id = label2id
model.config.id2label = id2label
model.config.num_labels = len(label_list)

model.save_pretrained("phobert-ner-test")
tokenizer.save_pretrained("phobert-ner-test")
print("Model initialized and saved!")

# Hàm tải dữ liệu
def load_data(file_path):
    sentences = []
    tags = []
    current_sentence = []
    current_tags = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == "[END]":
                if current_sentence:
                    sentences.append(current_sentence)
                    tags.append(current_tags)
                    current_sentence = []
                    current_tags = []
            elif line:
                token, tag = line.split()
                current_sentence.append(token)
                current_tags.append(tag)

    if current_sentence:
        sentences.append(current_sentence)
        tags.append(current_tags)

    return Dataset.from_dict({"tokens": sentences, "ner_tags": tags})

# Tải dữ liệu (giả sử file đã có sẵn)
dataset = load_data("/content/drive/MyDrive/Training-AI-materials/train.txt")
print(dataset)
print("Sample:", dataset[0])

# Hàm tokenize và căn chỉnh nhãn
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=64,  # Giảm xuống nếu dữ liệu ngắn
        return_offsets_mapping=False
    )

    labels = []
    for i, original_labels in enumerate(examples["ner_tags"]):
        input_ids = tokenized_inputs["input_ids"][i]
        tokens = tokenizer.convert_ids_to_tokens(input_ids)
        label_ids = []
        word_idx = 0

        for token in tokens:
            if token in ["[CLS]", "[SEP]", "[PAD]"]:
                label_ids.append(-100)
            elif token.startswith("##"):
                if word_idx < len(original_labels) and original_labels[word_idx].startswith("I-"):
                    label_ids.append(label2id[original_labels[word_idx]])
                else:
                    label_ids.append(-100)
            else:
                if word_idx < len(original_labels):
                    label_ids.append(label2id[original_labels[word_idx]])
                    word_idx += 1
                else:
                    label_ids.append(-100)

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Áp dụng tokenize
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)
print("Tokenized sample:", tokenized_dataset[0])

  Using cached underthesea-6.8.4-py3-none-any.whl.metadata (15 kB)
  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached python_crfsuite-0.9.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
  Using cached underthesea_core-1.0.4-cp311-cp311-manylinux2010_x86_64.whl.metadata (1.7 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

...

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Model initialized and saved!
Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 36
})
Sample: {'tokens': ['Sáng', 'ngày_mai', 'họp', '8h', 'trong_vòng', '30p'], 'ner_tags': ['B-TIME', 'I-TIME', 'B-EVENT', 'B-TIME', 'O', 'B-DURATION']}


Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Tokenized sample: {'tokens': ['Sáng', 'ngày_mai', 'họp', '8h', 'trong_vòng', '30p'], 'ner_tags': ['B-TIME', 'I-TIME', 'B-EVENT', 'B-TIME', 'O', 'B-DURATION'], 'input_ids': [0, 1316, 5183, 673, 11089, 40201, 323, 5315, 2380, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [1, 2, 3, 1, 0, 5, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -

In [ ]:
!pip install transformers datasets accelerate -U

In [ ]:
# Chia tập dữ liệu (tùy chọn)
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./phobert-ner-results",  # Thư mục lưu kết quả
    evaluation_strategy="epoch",         # Đánh giá sau mỗi epoch (nếu có eval_dataset)
    save_strategy="epoch",               # Lưu mô hình sau mỗi epoch
    learning_rate=2e-5,                  # Tốc độ học
    per_device_train_batch_size=16,      # Kích thước batch cho train
    per_device_eval_batch_size=16,       # Kích thước batch cho eval
    num_train_epochs=3,                  # Số epoch huấn luyện
    weight_decay=0.01,                   # Giảm trọng số để tránh overfitting
    load_best_model_at_end=True,         # Tải mô hình tốt nhất sau khi train
    logging_dir="./logs",                # Thư mục lưu log
    logging_steps=10,                    # Ghi log sau mỗi 10 bước
    report_to="none"
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
!pip install seqeval evaluate

import numpy as np
import evaluate

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Loại bỏ các giá trị -100 (token đặc biệt)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,                         # Mô hình đã khởi tạo
    args=training_args,                  # Tham số huấn luyện
    train_dataset=train_dataset,         # Tập dữ liệu huấn luyện
    eval_dataset=eval_dataset if 'eval_dataset' in locals() else None,  # Tập đánh giá (nếu có)
    tokenizer=tokenizer,                 # Tokenizer
    compute_metrics=compute_metrics,     # Hàm tính metrics
)

# Bắt đầu huấn luyện
trainer.train()

<ipython-input-14-09c00c9d643b>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.839350,0.172414,0.250000,0.204082,0.307692
2,No log,1.778348,0.157895,0.150000,0.153846,0.326923
3,No log,1.755691,0.111111,0.100000,0.105263,0.326923


TrainOutput(global_step=6, training_loss=1.8104902903238933, metrics={'train_runtime': 108.3846, 'train_samples_per_second': 0.775, 'train_steps_per_second': 0.055, 'total_flos': 2743739970048.0, 'train_loss': 1.8104902903238933, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/phobert-ner-trained")
tokenizer.save_pretrained("/content/drive/MyDrive/phobert-ner-trained")
print("Trained model saved!")

Trained model saved!


# Test Inference

In [ ]:
from transformers import pipeline

ner_pipeline = pipeline("ner", model="/content/drive/MyDrive/phobert-ner-trained", tokenizer=tokenizer)
test_sentence = "Sáng ngày_mai họp 8h trong_vòng 30p"
results = ner_pipeline(test_sentence.split())
print(results)

Exception ignored in: <function _xla_gc_callback at 0x7a3184132660>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[[], [], [], [{'entity': 'B-EVENT', 'score': np.float32(0.2124369), 'index': 1, 'word': '8h', 'start': None, 'end': None}], [{'entity': 'B-EVENT', 'score': np.float32(0.29482636), 'index': 2, 'word': 'vòng', 'start': None, 'end': None}], []]


In [ ]:
from transformers import pipeline

ner_pipeline = pipeline("ner", model="/content/drive/MyDrive/phobert-ner-trained", tokenizer="/content/drive/MyDrive/phobert-ner-trained")
test_sentence = "Sáng ngày_mai họp 8h trong_vòng 30p"
results = ner_pipeline(test_sentence)
print(results)

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'I-TIME', 'score': np.float32(0.1888013), 'index': 3, 'word': 'họp', 'start': None, 'end': None}, {'entity': 'B-EVENT', 'score': np.float32(0.22635457), 'index': 6, 'word': 'vòng', 'start': None, 'end': None}, {'entity': 'B-EVENT', 'score': np.float32(0.19132863), 'index': 8, 'word': 'p', 'start': None, 'end': None}]


# Continue traing

In [ ]:
# !pip install transformers==4.38.0 accelerate==0.26.0
# !pip install seqeval evaluate datasets underthesea
# !pip install -U transformers==4.41.2
# !pip install accelerate evaluate datasets seqeval underthesea

# 1. Gỡ transformers cũ hoàn toàn
# !pip uninstall -y transformers

# 2. Cài phiên bản mới (>= 4.41.0 để có EncoderDecoderCache)
# !pip install --upgrade transformers
!pip install transformers==4.40

# 3. (Tùy chọn) Cài các thư viện khác nếu cần
!pip install accelerate datasets evaluate seqeval underthesea


  Using cached transformers-4.38.0-py3-none-any.whl.metadata (131 kB)
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.38.0-py3-none-any.whl (8.5 MB)
Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.0 which is incompatible.


^C


In [ ]:
from datasets import Dataset

def load_data(file_path):
    sentences = []
    tags = []
    current_sentence = []
    current_tags = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == "[END]":
                if current_sentence:
                    sentences.append(current_sentence)
                    tags.append(current_tags)
                    current_sentence = []
                    current_tags = []
            elif line:
                token, tag = line.split()
                current_sentence.append(token)
                current_tags.append(tag)

    if current_sentence:
        sentences.append(current_sentence)
        tags.append(current_tags)

    return Dataset.from_dict({"tokens": sentences, "ner_tags": tags})

dataset = load_data("/content/drive/MyDrive/Training-AI-materials/train.txt")
print("Updated dataset:", dataset[1])


Updated dataset: {'tokens': ['10', 'phút', 'nữa', 'di_chơi'], 'ner_tags': ['B-TIME', 'I-TIME', 'I-TIME', 'B-EVENT']}


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Tải tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/phobert-ner-trained")

# Tải mô hình
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/phobert-ner-trained")

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        padding="max_length",
        truncation=True,
        max_length=64,
        return_offsets_mapping=False,
        add_special_tokens=True,
    )

    labels = []
    for i in range(len(examples["tokens"])):
        word_list = examples["tokens"][i]
        label_list = examples["ner_tags"][i]
        input_ids = tokenized_inputs["input_ids"][i]
        tokens = tokenizer.convert_ids_to_tokens(input_ids)

        label_ids = []
        word_idx = 0
        subword_accumulator = ""

        for token in tokens:
            if token in tokenizer.all_special_tokens:
                label_ids.append(-100)
            elif token.endswith("@@"):
                subword_accumulator += token.replace("@@", "")
                # vẫn là subword của từ hiện tại → không gán nhãn
                label_ids.append(-100)
            else:
                subword_accumulator += token
                if word_idx < len(label_list):
                    label_ids.append(model.config.label2id[label_list[word_idx]])
                    word_idx += 1
                else:
                    label_ids.append(-100)
                subword_accumulator = ""

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


# Áp dụng tokenize cho dữ liệu
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [ ]:
#Kiểm tra hàm tokenize_and_align_labels
from pprint import pprint

# Gói sample lại thành list để giả lập batch
sample_batch = {"tokens": [dataset[0]["tokens"]], "ner_tags": [dataset[0]["ner_tags"]]}
tokenized = tokenize_and_align_labels(sample_batch)

tokens = tokenizer.convert_ids_to_tokens(tokenized['input_ids'][0])
labels = tokenized['labels'][0]

pprint(list(zip(tokens, labels)))


[('<s>', -100),
 ('Sáng', 1),
 ('ngày_mai', 2),
 ('họp', 3),
 ('8h', 1),
 ('trong_@@', -100),
 ('vòng', 0),
 ('30@@', -100),
 ('p', 5),
 ('</s>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 ('<pad>', -100),
 

In [ ]:
label_map = model.config.id2label

for tok, lid in zip(tokens, labels):
    label_name = label_map[lid] if lid != -100 else "IGN"
    print(f"{tok:<10} → {label_name}")


<s>        → IGN
Sáng       → B-TIME
ngày_mai   → I-TIME
họp        → B-EVENT
8h         → B-TIME
trong_@@   → IGN
vòng       → O
30@@       → IGN
p          → B-DURATION
</s>       → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → IGN
<pad>      → 

In [ ]:
# check lỗi file data
def find_invalid_labels_verbose(file_path, valid_labels):
    with open(file_path, 'r', encoding='utf-8') as f:
        sentence_idx = 0
        line_idx = 0
        current_sentence = []

        for line in f:
            line_idx += 1
            line = line.strip()
            if line == "[END]":
                print(f"✅ [END] tại dòng {line_idx} — kết thúc câu {sentence_idx}")
                sentence_idx += 1
                current_sentence = []
            elif line:
                parts = line.split()
                if len(parts) != 2:
                    print(f"❌ Lỗi định dạng tại dòng {line_idx}: '{line}'")
                    continue

                token, label = parts
                if label not in valid_labels:
                    print(f"❌ Nhãn không hợp lệ tại dòng {line_idx} (câu {sentence_idx}): '{label}'")
                else:
                    print(f"✅ Dòng {line_idx}: Token = '{token}', Label = '{label}' — Hợp lệ")

#valid_labels = set(model.config.label2id.keys())
#find_invalid_labels_verbose("/content/drive/MyDrive/Training-AI-materials/train.txt", valid_labels)


In [ ]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./phobert-ner-results-continued",  # Thư mục lưu kết quả mới
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,                    # Có thể giảm learning rate để tinh chỉnh
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,                    # Số epoch mới để huấn luyện thêm
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

In [ ]:
#!pip install seqeval evaluate

import numpy as np
import evaluate

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Loại bỏ các giá trị -100 (token đặc biệt)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,                         # Mô hình đã khởi tạo
    args=training_args,                  # Tham số huấn luyện
    train_dataset=train_dataset,         # Tập dữ liệu huấn luyện
    eval_dataset=eval_dataset if 'eval_dataset' in locals() else None,  # Tập đánh giá (nếu có)
    tokenizer=tokenizer,                 # Tokenizer
    compute_metrics=compute_metrics,     # Hàm tính metrics
)

label_list = ["O", "B-TIME", "I-TIME", "B-EVENT", "I-EVENT", "B-DURATION", "I-DURATION"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

# Bắt đầu huấn luyện
trainer.train()